# Prediction notebook for Yolo ensembling 

This notebook is designed for predictions using ensembling. You can adjust the model paths or modify the model combinations to experiment with your own models. For a detailed explanation of the inference process, refer to the standalone inference.ipynb file.

In [1]:
import numpy as np
import sys
import pandas as pd
from tqdm import tqdm
import shutil
import time

sys.path.append("../")
from detection import YoloInference,EnsembleYolo
from notebooks.utils_notebook import filter_boxes

In [ ]:
itos={0:'B', 1:'BA', 2:'EO', 3:'Er', 4:'LAM3', 5:'LF', 6:'LGL', 7:'LH_lyAct', 8:'LLC', 9:'LM', 10:'LY', 11:'LZMG', 12:'LyB', 13:'Lysee', 14:'M', 15:'MBL', 16:'MM', 17:'MO', 18:'MoB', 19:'PM', 20:'PNN', 21:'SS', 22:'Thromb'}

print_inference_time=False
data_path="../data/Cytologia/images"


test_csv_path="../data/Cytologia/test.csv"
csv_path="../data/Cytologia/predictions.csv"
shutil.copy(test_csv_path,"../data/Cytologia/predictions.csv")

df = pd.read_csv(csv_path)


yolo_engine1=YoloInference("../models/best/yolo11n384_blk.pt",device="cuda",load_model=True) # load_model=False => add this parameter to avoid loading the model if your computer has limited memory
yolo_engine2=YoloInference("../models/best/yolo11m384_iou.pt",device="cuda",load_model=True) 
yolo_engine3=YoloInference("../models/best/yolov10n384_blk.pt",device="cuda",load_model=True) 
yolo_engine4=YoloInference("../models/best/yolov10m384_iou.pt",device="cuda",load_model=True)
yolo_engine5=YoloInference("../models/best/yolo11x384_iou.pt",device="cuda",load_model=True) 
yolo_engine6=YoloInference("../models/best/yolov10s384_iou.pt",device="cuda",load_model=True) 
yolo_engine7=YoloInference("../models/best/yolo11n384_nc.pt",device="cuda",load_model=True) 

ens_engine=EnsembleYolo([yolo_engine1,yolo_engine2,yolo_engine3,yolo_engine4,yolo_engine5,yolo_engine6,yolo_engine7],use_probs=False)

# Add missing columns
if not {'x1', 'y1', 'x2', 'y2', 'class'}.issubset(df.columns):
    for col in ['x1', 'y1', 'x2', 'y2', 'class']:
        df[col] = np.nan

# Get unique NAME to infer each image only once
names = df["NAME"].unique()
tqdm_names= tqdm(names)
no_dets=0
for name in tqdm_names:
    img_path=f"{data_path}/{name}"

    occurences = df[df["NAME"]==name]
    trustii_ids = occurences["trustii_id"].tolist()
    begin = time.time()
    boxes,scores,labels=ens_engine.predict(img_path,conf=0.00001,verbose=False) 
    
    boxes=list(boxes)
    scores=list(scores)
    labels=list(labels)
    
    if len(occurences) < len(boxes):
        boxes,scores,labels=filter_boxes(occurences,boxes,scores,labels)

    for idx,(box,label) in enumerate(zip(boxes,labels)):
        x1,y1,x2,y2 = box
        trustii_id = trustii_ids[idx]
        cls=itos[label]
        df.loc[df["trustii_id"] == trustii_id, ["x1", "y1", "x2", "y2", "class"]] = [x1, y1, x2, y2, cls]

    # case where a wbc is not detected (even with this very low threshold)
    for i in range(len(boxes),len(occurences)):
        trustii_id = trustii_ids[i]
        df.loc[df["trustii_id"] == trustii_id, ["x1", "y1", "x2", "y2", "class"]] = [0, 0, 0, 0, 'PNN']

    end=time.time()
    if print_inference_time:
        print(f"Time for inference: {end-begin}")
print("Number of undetected WBCs: ",no_dets)
df.to_csv(csv_path, index=False)
print("CSV mis à jour avec succès.")


  0%|          | 0/20751 [00:00<?, ?it/s]/tmp/ipykernel_7412/3418722514.py:52: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'LLC' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[df["trustii_id"] == trustii_id, ["x1", "y1", "x2", "y2", "class"]] = [x1, y1, x2, y2, cls]
  0%|          | 3/20751 [00:01<1:39:20,  3.48it/s]

Time for inference: 0.9012157917022705
Time for inference: 0.06791305541992188
Time for inference: 0.0720376968383789


  0%|          | 5/20751 [00:01<1:00:52,  5.68it/s]

Time for inference: 0.07290077209472656
Time for inference: 0.06574821472167969
Time for inference: 0.0740971565246582


  0%|          | 9/20751 [00:01<38:20,  9.01it/s]  

...

  1%|          | 153/20751 [00:13<23:31, 14.59it/s]

Time for inference: 0.06385445594787598
Time for inference: 0.0650320053100586
Time for inference: 0.062444210052490234
Time for inference: 0.06280040740966797


  1%|          | 156/20751 [00:13<29:53, 11.48it/s]

Time for inference: 0.19744253158569336
Time for inference: 0.07164645195007324


KeyboardInterrupt: 